In [ ]:
from api_and_repo_link import link_me_up
import api_and_repo_link
from importlib import reload
reload(api_and_repo_link)
link_me_up()

In [ ]:

from MAPPINGS import COUNTRY_VALUE_MAP, STATUS_VALUE_MAP, STATUS_VALUE_MAP_CSV, SUBSIDIARY_VALUE_MAP, SUBSIDIARY_VALUE_MAP_CSV, CURRENCY_VALUE_MAP, COLUMN_MAPPING, COLUMN_MAPPING_ADDRESS_BOOK, COLUMN_MAPPING_ADDRESS

import pandas
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite

In [ ]:
from multithread_uploader import upload_all_these_records
api = APIConfig_netsuite()
from util.get_ns_number import get_ns_number
Customer = api.client.get_type("ns13:Customer")
get_ns_number(api, "Customer")

In [35]:
def do_value_mappings(old_column_name, customer_dict):
    value = customer_dict[old_column_name]
    # this werid character \xa0 is removed if found...!
    if type(value) == str:
        value = value.replace("\xa0", " ")

    if old_column_name.endswith("_country"):
        return COUNTRY_VALUE_MAP[value]
    elif old_column_name == "currency":
        return CURRENCY_VALUE_MAP[value]
    elif old_column_name == "status":
        return STATUS_VALUE_MAP[STATUS_VALUE_MAP_CSV[value]]
    elif old_column_name == "subsidiary":
        return SUBSIDIARY_VALUE_MAP[SUBSIDIARY_VALUE_MAP_CSV[value]]
    else:
        return value

def to_dict(customer_series):
    customer_series[customer_series.isna()] = None # might work!
    customer_dict = customer_series.to_dict()

    # deal with value mappings here straight away
    for oldCol in customer_dict.keys():
        customer_dict[oldCol] = do_value_mappings(oldCol, customer_dict)

    # now deal with the column mappings
    for oldCol, newCol in COLUMN_MAPPING.items():
        customer_dict[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    addressbookList = {
        "addressbook": [
            {
                "addressbookAddress": dict(),
            },
            {
                "addressbookAddress": dict(),
            },
        ]
    }

    Addbook1 = addressbookList["addressbook"][0]
    Addbook2 = addressbookList["addressbook"][1]
    Address1 = addressbookList["addressbook"][0]["addressbookAddress"]
    Address2 = addressbookList["addressbook"][1]["addressbookAddress"]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS_BOOK.items():
        Addbook = Addbook1 if oldCol.startswith("Address1_") else Addbook2

        Addbook[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS.items():
        Address = Address1 if oldCol.startswith("Address1_") else Address2

        Address[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    customer_dict["addressbookList"] = addressbookList
    
    customer_dict["entityId"] = "customer_human_" + str(customer_dict["entityId"])
    customer_dict["externalId"] = "customer_human_" + str(customer_dict["externalId"])

    return customer_dict

In [36]:
test_df = pandas.read_csv("test_input_dummy_customers.csv")

test_list = [Customer(**to_dict(row)) for index, row in test_df.iterrows()]
test_list

[{
     'nullFieldList': None,
     'customForm': None,
     'entityId': 'customer_human_10787123',
     'altName': None,
     'isPerson': True,
     'phoneticName': None,
     'salutation': None,
     'firstName': 'Spangle',
     'middleName': None,
     'lastName': 'Flooper',
     'companyName': None,
     'entityStatus': {
         'name': None,
         'internalId': 13,
         'externalId': None,
         'type': None
     },
     'parent': None,
     'phone': None,
     'fax': None,
     'email': 'example123@hotmail.com',
     'url': None,
     'defaultAddress': None,
     'isInactive': None,
     'category': None,
     'title': None,
     'printOnCheckAs': None,
     'altPhone': None,
     'homePhone': None,
     'mobilePhone': None,
     'altEmail': None,
     'language': None,
     'comments': None,
     'numberFormat': None,
     'negativeNumberFormat': None,
     'dateCreated': None,
     'image': None,
     'emailPreference': None,
     'subsidiary': {
         'name': No

In [37]:
import multithread_uploader
reload(multithread_uploader)
from multithread_uploader import upload_all_these_records
upload_all_these_records(test_list, api, "addList")

thread (0) start_row: 0, end_row: 1
thread (1) start_row: 1, end_row: 2
thread (2) start_row: 2, end_row: 3
thread (3) start_row: 3, end_row: 4
+++++++++ new thread (0) +++++++++
+++++++++ new thread (2) +++++++++
+++++++++ new thread (1) +++++++++
+++++++++ new thread (3) +++++++++
> maybe uploaded [1] records
0:00:07.190571
> maybe uploaded [2] records
0:00:07.197571
+++++++++ thread completed (1) +++++++++
+++++++++ thread completed (0) +++++++++
> maybe uploaded [3] records
0:00:07.327448
> maybe uploaded [4] records
0:00:07.335426
+++++++++ thread completed (3) +++++++++
+++++++++ thread completed (2) +++++++++


In [38]:
import os
pandas.concat(
    [
        pandas.read_csv("thread_logs/" + filename)
        for filename in os.listdir("thread_logs")
    ]
)

,key_0,afterSubmitFailed,code,message,num_status_details,success,type,accessRole,accountNumber,addressbookList,...,thirdPartyAcct,thirdPartyCountry,thirdPartyZipcode,title,unbilledOrders,url,vatRegNumber,visits,webLead,api_method
0,0,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
1,0,0.0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
0,0,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
1,0,0.0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
0,0,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
1,0,0.0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
0,0,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
1,0,0.0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
